In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import scipy.signal as signal

In [2]:
# PURPOSE: Convert a text string to a stream of 1/0 bits.
# INPUT: string message
# OUTPUT: vector of 1's and 0's.
def text2bits(message):
    # Convert to characters of '1' and '0' in a vector.
    temp_message = []
    final_message = []
    for each in message:
        temp_message.append(format(ord(each), '07b'))
    for every in temp_message:
        for digit in every:
            final_message.append(int(digit))
    return final_message

In [3]:
pseudonym_mesage = 'STOP'
pseudonym_packet = text2bits(pseudonym_mesage)

In [4]:
'''
Protocol parameters!!!
'''
FFT = 64 # FFT size for extracting IQ sample in each subcarrier
OFDM_size = 80 # OFDM symbol with cyclic prefix
data_size = 48 # data_subcarriers
mess_length = 480
actual_message = 'I have done my PhD in electrical engineering at WashU. What a journey it has been!'

82

In [5]:
packet = 6000 # number of bits per pseudonym bit
samples = packet//10 # number of samples per chip
CP = 16
mod_index = 1.0 # decides the level of modification/modulation on the host signal

In [6]:
CP = FFT//4  # 25% cyclic prefix
pilotValue = 2.83+2.83j # known values for pilots
# pilotValue = 1.4142+1.4142j # known values for pilots
pseudonymValue = 1.4142+1.4142j

In [ ]:
allCarriers = np.array([-32,-31,-30,-29,-28,-27,-26,-25,-24,-23,-22,-21,-20,-19,-18,-17,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,
                    -5,-4,-3,-2,-1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31])
print('Length of allCarriers',len(allCarriers))
dataCarriers = np.array([-26,-25,-24,-23,-22,-20,-19,-18,-17,-16,-15,-14,-13,-12,-11,-10,-9,-8,-6,
                    -5,-4,-3,-2,-1,0,1,2,3,4,5,6,8,9,10,11,12,13,14,15,16,17,18,19,20,22,23,24,25])
pseudonymCarrier = np.array([26])
pilotCarriers = np.array([-21,-7,7,21])
guardCarriers = np.array([-32,-31,-30,-29,-28,-27,27,28,29,30,31])

plt.plot(pilotCarriers, np.zeros_like(pilotCarriers), 'bo', label='pilot subcarriers')
plt.plot(dataCarriers, np.zeros_like(dataCarriers), 'ro', label='data subcarriers')
plt.plot(guardCarriers, np.zeros_like(guardCarriers), 'ko', label='guard subcarriers')
plt.plot(pseudonymCarrier, np.zeros_like(pseudonymCarrier), 'yo', label='pseudonym subcarrier')
plt.legend()

In [8]:
# PURPOSE: convert input data stream to signal space values for
#          a particular modulation type (as specified by the inputVec
#          and outputVec).
# INPUT: data (groups of bits)
# OUTPUT: signal space values
def lut(data, inputVec, outputVec):
    if len(inputVec) != len(outputVec):
        print('Input and Output vectors must have identical length')
    # Initialize output
    output = np.zeros(data.shape)
    # For each possible data value
    eps = np.finfo('float').eps
    for i in range(len(inputVec)):
        # Find the indices where data is equal to that input value
        for k in range(len(data)):
            if abs(data[k]-inputVec[i]) < eps:
                # Set those indices in the output to be the appropriate output value.
                output[k] = outputVec[i]
    return output

In [9]:
# PURPOSE: Convert binary data to M-ary by making groups of log2(M)
#          bits and converting each bit to one M-ary digit.
# INPUT: Binary digit vector, with length as a multiple of log2(M)
# OUTPUT: M-ary digit vector
def binary2mary(data, M):
    length     = len(data)
    log2M   = round(math.log2(M)) # integer number of bits per group
    if (length % log2M) != 0:
        print('Input to binary2mary must be divisible by log2(m).')
    binvalues = np.zeros((log2M,1))
    values = []
    newdata = []
    start = log2M-1
    i = 0
    while start >= 0:
        binvalues[i] = int(math.pow(2,start))
        start=start-1
        i = i + 1
    for each in data:
        newdata.append(int(each))
    newdata = np.array(newdata)
    temp = np.reshape(newdata, (int(length/log2M), log2M))
    marydata = temp.dot(binvalues)
    return marydata

In [10]:
# Signal Generation
# INPUT: none
# OUTPUT: binary data
def Data_Frame(x):
    A = math.sqrt(9/2)
    plt.ion()

    if 1:
        # print('Lenght of message:',len(x))
        data_bits = x
        # print('Number of data bits: ' + str(len(data_bits)))

        ###########################################
        ### Signal Generation
        ### INPUT: binary data
        ### OUTPUT: 4-ary data (0..3) values

        data = binary2mary(data_bits, 4)

        ###########################################
        ### Modulation
        ### INPUT: data
        ### OUTPUT: modulated values, x

        inputVec   = [0, 1, 2, 3]
        outputVecI = [A, -A, A, -A]
        outputVecQ = [A, A, -A, -A]
        xI         = lut(data, inputVec, outputVecI)
        xQ         = lut(data, inputVec, outputVecQ)
        xI = xI.reshape((1,len(data)))
        xQ = xQ.reshape((1,len(data)))

        x_s_I = np.reshape(xI, xI.size)
        x_s_Q = np.reshape(xQ, xQ.size)

        qpsk_IQ = x_s_I + 1j*x_s_Q

#         print("Length of QPSK_IQ:",len(qpsk_IQ))
#         plt.figure()
#         plt.plot(np.real(qpsk_IQ),label='Real Signal')
#         plt.plot(np.imag(qpsk_IQ),label='Imag Signal')
#         plt.grid('on')
#         plt.legend()
#         plt.show()
    return qpsk_IQ

# Generate Transmit Bits

In [12]:
def Generate_QPSK_Samples(x):
    add_bits = np.array([1,0])
    TX_bits = np.concatenate([text2bits(x),add_bits])
    QPSK_signal = Data_Frame(TX_bits)
    data_frame = np.tile(QPSK_signal,100)
    
    return data_frame

In [13]:
''' Generate the OFDM time signal for the input QPSK signal'''

def High_Amplitude(data):
    for i in range(len(data)//data_size):
        
        QPSK_payload = data[i*data_size:(i+1)*data_size]
        
        symbol = np.zeros(FFT, dtype=complex) # the overall K subcarriers
        symbol[pilotCarriers] = pilotValue  # allocate the pilot subcarriers 
        symbol[dataCarriers] = QPSK_payload  # allocate the data subcarriers
        symbol[pseudonymCarrier] = pseudonymValue*(1+mod_index) # Generate (1+m) times the original amplitude of the QPSK signal
        
        # Generate time domain signal
        OFDM_time = np.fft.ifft(symbol,n=64)

        # adding cyclic prefix
        def addCP(x):
            cp = x[-CP:]              
            return np.hstack([cp, x]) 
        OFDM_withCP = addCP(OFDM_time)
        if i == 0:
            OFDM_swap = OFDM_withCP
        else:
            OFDM_signal = np.hstack([OFDM_swap, OFDM_withCP])
            OFDM_swap = OFDM_signal   
    return OFDM_signal

In [14]:
''' Generate the OFDM time signal for the input QPSK signal'''

def Low_Amplitude(data):
    for i in range(len(data)//data_size):
        
        QPSK_payload = data[i*data_size:(i+1)*data_size]
        
        symbol = np.zeros(FFT, dtype=complex) # the overall K subcarriers
        symbol[pilotCarriers] = pilotValue  # allocate the pilot subcarriers 
        symbol[dataCarriers] = QPSK_payload  # allocate the data subcarriers
        symbol[pseudonymCarrier] = pseudonymValue*(1-mod_index) # Generate (1+m) times the original amplitude of the QPSK signal
        
        # Generate time domain signal
        OFDM_time = np.fft.ifft(symbol,n=64)

        # adding cyclic prefix
        def addCP(x):
            cp = x[-CP:]              
            return np.hstack([cp, x]) 
            
        OFDM_withCP = addCP(OFDM_time)
        if i == 0:
            OFDM_swap = OFDM_withCP
        else:
            OFDM_signal = np.hstack([OFDM_swap, OFDM_withCP])
            OFDM_swap = OFDM_signal
            
    return OFDM_signal

In [15]:
'''Embedding the pseudonym watermark signal on to the OFDM host signal
Divide the ofdm packet into 10 window of samples of size 600 called chips
Send chips with alternative high and low power patterns which can be detected 
as power changes at the passive receiver.'''

def Watermarking(x):
    for i in range(len(pseudonym_packet)):
        if pseudonym_packet[i] ==1:
            swap = High_Amplitude(Generate_QPSK_Samples(x))
            for j in range(1,10):
                if j%2 !=0:
                    watermark_signal = np.concatenate([swap,Low_Amplitude(Generate_QPSK_Samples(x))])
                else:
                    watermark_signal = np.concatenate([swap,High_Amplitude(Generate_QPSK_Samples(x))])
                swap = watermark_signal
       
        else:
            swap = Low_Amplitude(Generate_QPSK_Samples(x))
            for k in range(1,10):
                if k%2 !=0:
                    watermark_signal = np.concatenate([swap,High_Amplitude(Generate_QPSK_Samples(x))])  
                else:
                    watermark_signal = np.concatenate([swap,Low_Amplitude(Generate_QPSK_Samples(x))])
                swap = watermark_signal 
        if i == 0:
            SWAP = watermark_signal 
        else:
            watermark = np.concatenate([SWAP,watermark_signal])
            SWAP = watermark
    
    return watermark

In [16]:
# Generate HTSTF signals for time synchronization
def Generate_HTSTF():
    data = scipy.io.loadmat('HTSTF.mat')
    new_data = data['stf'].reshape((1,len(data['stf'])))
    preamble = np.tile(new_data,10)[0]
    return preamble

In [17]:
''' Generate the watermark TX signal '''
def Generate_OFDM_Packet(x):
    
    # Generate the watermark signal
    watermarked_OFDM_signal = Watermarking(x)
    
    # create a preamble for synchronization
    OFDM_preamb =  Generate_HTSTF()
    
    # Concatenate preamble signal to OFDM_data
    packet = np.concatenate((OFDM_preamb,2.5*watermarked_OFDM_signal))
    
    return packet

In [ ]:
TX_packet = Generate_OFDM_Packet(actual_message)

plt.plot(abs(TX_packet[:6800]))
print('Length of OFDM packet',len(TX_packet))
print('Max power in dBm',10*np.log10(1.414*np.mean(abs(TX_packet)**2)))

In [ ]:
plt.psd(TX_packet)
plt.show()

In [22]:
# The SDR transmits IQ samples as IQIQIQ... patern 
# Convert complex to binary
def write_complex_binary(data, filename):
    '''
    Open filename and write array to it as binary
    Format is interleaved float IQ e.g. each I,Q should be 32-bit float 
    INPUT
    ----
    data:     data to be wrote into the file. format: (length, )
    filename: file name
    '''
    re = np.real(data)
    im = np.imag(data)
    binary = np.zeros(len(data)*2, dtype=np.float32)
    binary[::2] = re
    binary[1::2] = im
    binary.tofile(filename) 

In [23]:
# save generated signal to file
write_complex_binary(TX_packet, 'SingleChannel_watermark_Aug9_2024.iq')